In [1]:
from IPython import get_ipython
%load_ext autoreload
%autoreload 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
import os
device_mappings = {0: 1, 1: 5, 2: 6, 3: 7, 4: 2, 5: 3, 6: 0, 7: 4}
os.environ["CUDA_VISIBLE_DEVICES"] = str(device_mappings[6])

In [3]:
os.environ['TRANSFORMERS_CACHE'] = '/nas/xd/.cache/torch/transformers'

In [42]:
import torch
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.sampler import RandomSampler, Sampler, SequentialSampler

from transformers import (
    CONFIG_MAPPING,
    MODEL_WITH_LM_HEAD_MAPPING,
    AutoConfig,
    AutoModelWithLMHead,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    HfArgumentParser,
    LineByLineTextDataset,
    PreTrainedTokenizer,
    TextDataset,
    Trainer,
    TrainingArguments,
    set_seed,
)

In [7]:
from run_language_modeling import DataTrainingArguments, get_dataset

In [4]:
import logging
import os

logging.basicConfig(level=os.environ.get("LOGLEVEL", "WARNING"))

In [6]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [7]:
tokenizer = T5Tokenizer.from_pretrained('t5-3b')

In [8]:
proxies = {'https': '192.168.50.1:1081'}
model = T5ForConditionalGeneration.from_pretrained('t5-3b', proxies=proxies)


loading state_dict took 189.682 sec


In [51]:
config11b = AutoConfig.from_pretrained('/nas/xd/.cache/torch/transformers/t5-11b-config.json')

In cached_path: url_or_filename = /nas/xd/.cache/torch/transformers/t5-11b-config.json


In [52]:
 model11b = T5ForConditionalGeneration.from_pretrained('/nas/xd/.cache/torch/transformers/t5-11b-pytorch_model.bin', config=config11b)

In cached_path: url_or_filename = /nas/xd/.cache/torch/transformers/t5-11b-pytorch_model.bin
loading state_dict took 750.661 sec


In [63]:
model = model11b

In [116]:
text = 'Z is larger than Z? <extra_id_0> Z is smaller. </s>'
input_ids = tokenizer(text, return_tensors="pt").input_ids
print(tokenizer.convert_ids_to_tokens(input_ids[0]))
# outputs = model.generate(input_ids)
# tokenizer.decode(outputs[0])

['▁Z', '▁is', '▁larger', '▁than', '▁Z', '?', '<extra_id_0>', '▁Z', '▁is', '▁smaller', '.', '</s>']


In [127]:
examples = [('Z is smaller than X? <extra_id_0> Z is smaller. </s>', 'Right'),
 ('Z is smaller than X? <extra_id_0> X is larger. </s>', 'Right'),
 ('X is larger than Z? <extra_id_0> Z is smaller. </s>', 'Right'),
 ('X is larger than Z? <extra_id_0> X is larger. </s>', 'Right'),
 ('Z is smaller than X? <extra_id_0> X is smaller. </s>', 'Wrong'),
 ('Z is smaller than X? <extra_id_0> Z is larger. </s>', 'Wrong'),
 ('X is larger than Z? <extra_id_0> X is smaller. </s>', 'Wrong'),
 ('X is larger than Z? <extra_id_0> Z is larger. </s>', 'Wrong')]
texts, labels = zip(*examples)
texts

('Z is smaller than X? <extra_id_0> Z is smaller. </s>',
 'Z is smaller than X? <extra_id_0> X is larger. </s>',
 'X is larger than Z? <extra_id_0> Z is smaller. </s>',
 'X is larger than Z? <extra_id_0> X is larger. </s>',
 'Z is smaller than X? <extra_id_0> X is smaller. </s>',
 'Z is smaller than X? <extra_id_0> Z is larger. </s>',
 'X is larger than Z? <extra_id_0> X is smaller. </s>',
 'X is larger than Z? <extra_id_0> Z is larger. </s>')

In [129]:
inputs = tokenizer.batch_encode_plus(texts[:], add_special_tokens=False, padding=True, return_tensors='pt')
# inputs['input_ids'].size()
# inputs['input_ids']
# inputs['attention_mask']
input_ids = inputs['input_ids']
for input_id in input_ids: print(tokenizer.convert_ids_to_tokens(input_id))

['▁Z', '▁is', '▁smaller', '▁than', '▁', 'X', '?', '<extra_id_0>', '▁Z', '▁is', '▁smaller', '.', '</s>', '<pad>']
['▁Z', '▁is', '▁smaller', '▁than', '▁', 'X', '?', '<extra_id_0>', '▁', 'X', '▁is', '▁larger', '.', '</s>']
['▁', 'X', '▁is', '▁larger', '▁than', '▁Z', '?', '<extra_id_0>', '▁Z', '▁is', '▁smaller', '.', '</s>', '<pad>']
['▁', 'X', '▁is', '▁larger', '▁than', '▁Z', '?', '<extra_id_0>', '▁', 'X', '▁is', '▁larger', '.', '</s>']
['▁Z', '▁is', '▁smaller', '▁than', '▁', 'X', '?', '<extra_id_0>', '▁', 'X', '▁is', '▁smaller', '.', '</s>']
['▁Z', '▁is', '▁smaller', '▁than', '▁', 'X', '?', '<extra_id_0>', '▁Z', '▁is', '▁larger', '.', '</s>', '<pad>']
['▁', 'X', '▁is', '▁larger', '▁than', '▁Z', '?', '<extra_id_0>', '▁', 'X', '▁is', '▁smaller', '.', '</s>']
['▁', 'X', '▁is', '▁larger', '▁than', '▁Z', '?', '<extra_id_0>', '▁Z', '▁is', '▁larger', '.', '</s>', '<pad>']


In [144]:
outputs = model.generate(**inputs, max_length=6)

In [146]:
outputs

tensor([[    0, 32099,  2163,     6, 32098,     3],
        [    0, 32099,   465,     6, 32098,   145],
        [    0, 32099,    37,    29, 32098,     3],
        [    0, 32099,    37,    29, 32098,     3],
        [    0, 32099,   465,     6, 32098,    19],
        [    0, 32099,   465,     6, 32098,   145],
        [    0, 32099,   465,     6, 32098,     3],
        [    0, 32099,   465,     6, 32098,     5]])

In [174]:
output = outputs[2]
output[0]
assert torch.all(outputs[:, 0] == tokenizer.pad_token_id)
assert torch.all(outputs[:, 1] == tokenizer.additional_special_tokens_ids[0])  # '<extra_id_0>'
assert (outputs == tokenizer.additional_special_tokens_ids[1]).sum() == outputs.size(0)

tensor(0)

In [175]:
tokenizer.decode(output[2:]).split(tokenizer.additional_special_tokens[1])[0]

'Then'

In [147]:
for text, label, output in zip(texts, labels, outputs):
    print(text, label, tokenizer.decode(output, clean_up_tokenization_spaces=False))
    print(tokenizer.convert_ids_to_tokens(output))

Z is smaller than X? <extra_id_0> Z is smaller. </s> Right <extra_id_0> Yes,<extra_id_1> 
['<pad>', '<extra_id_0>', '▁Yes', ',', '<extra_id_1>', '▁']
Z is smaller than X? <extra_id_0> X is larger. </s> Right <extra_id_0> No,<extra_id_1> than
['<pad>', '<extra_id_0>', '▁No', ',', '<extra_id_1>', '▁than']
X is larger than Z? <extra_id_0> Z is smaller. </s> Right <extra_id_0> Then<extra_id_1> 
['<pad>', '<extra_id_0>', '▁The', 'n', '<extra_id_1>', '▁']
X is larger than Z? <extra_id_0> X is larger. </s> Right <extra_id_0> Then<extra_id_1> 
['<pad>', '<extra_id_0>', '▁The', 'n', '<extra_id_1>', '▁']
Z is smaller than X? <extra_id_0> X is smaller. </s> Wrong <extra_id_0> No,<extra_id_1> is
['<pad>', '<extra_id_0>', '▁No', ',', '<extra_id_1>', '▁is']
Z is smaller than X? <extra_id_0> Z is larger. </s> Wrong <extra_id_0> No,<extra_id_1> than
['<pad>', '<extra_id_0>', '▁No', ',', '<extra_id_1>', '▁than']
X is larger than Z? <extra_id_0> X is smaller. </s> Wrong <extra_id_0> No,<extra_id_1> 
['<

In [49]:
model2 = T5ForConditionalGeneration.from_pretrained('t5-3b', proxies=proxies)

TRANSFORMERS_CACHE = /nas/xd/.cache/torch/transformers
In cached_path: url_or_filename = https://s3.amazonaws.com/models.huggingface.co/bert/t5-3b-config.json
In url_to_filename: url = https://s3.amazonaws.com/models.huggingface.co/bert/t5-3b-config.json
In url_to_filename: filename = 5eece39a4bfbb8cc0ffa95a83bb96cf22116bd653e3dc631f5a61cb7485b2231
In url_to_filename: filename.etag = 5eece39a4bfbb8cc0ffa95a83bb96cf22116bd653e3dc631f5a61cb7485b2231.330574dd7bece9f3ac99bd88cd0150f5bb560cf6a994df98965924d91cc87471
In cached_path: output_path = /nas/xd/.cache/torch/transformers/5eece39a4bfbb8cc0ffa95a83bb96cf22116bd653e3dc631f5a61cb7485b2231.330574dd7bece9f3ac99bd88cd0150f5bb560cf6a994df98965924d91cc87471
In cached_path: url_or_filename = https://cdn.huggingface.co/t5-3b-pytorch_model.bin
In url_to_filename: url = https://cdn.huggingface.co/t5-3b-pytorch_model.bin
In url_to_filename: filename = 17dc32cd6f6b528a7ab83521971a0ca1c81962ed03f703f88c6d7483f00f8d15
In url_to_filename: filename.et

In [44]:
model_name = 'gpt2'
config = AutoConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelWithLMHead.from_pretrained(model_name, config=config)
_ = model.eval()

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-config.json from cache at /home/xd/.cache/torch/transformers/4be02c5697d91738003fb1685c9872f284166aa32e061576bbe6aaeb95649fcf.db13c9bc9c7bdd738ec89e069621d88e05dc670366092d809a9cbcac6798e24e
INFO:transformers.configuration_utils:Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "resid_pdrop": 0.1,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true

In [3]:
data_args = DataTrainingArguments()
data_args.eval_data_file = '/nas/xd/data/wikitext-103-raw/wiki.test.raw.detok'
data_args.block_size = tokenizer.max_len
data_args

eval_dataset = get_dataset(data_args, tokenizer=tokenizer, evaluate=True)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=data_args.mlm, mlm_probability=data_args.mlm_probability)
dataloader = DataLoader(eval_dataset, sampler=SequentialSampler(eval_dataset), batch_size=1, collate_fn=data_collator)

In [43]:
for i, inputs in enumerate(dataloader):
    if i == 1: break
    print(i)
    print(tokenizer.decode(inputs['input_ids'][0]))

0

= Robert Boulter =

Robert Boulter is an English film, television and theatre actor. He had a guest-starring role on the television series The Bill in 2000. This was followed by a starring role in the play Herons written by Simon Stephens, which was performed in 2001 at the Royal Court Theatre. He had a guest role in the television series Judge John Deed in 2002. In 2004 Boulter landed a role as "Craig" in the episode "Teddy's Story" of the television series The Long Firm; he starred alongside actors Mark Strong and Derek Jacobi. He was cast in the 2005 theatre productions of the Philip Ridley play Mercury Fur, which was performed at the Drum Theatre in Plymouth and the Menier Chocolate Factory in London. He was directed by John Tiffany and starred alongside Ben Whishaw, Shane Zaza, Harry Kent, Fraser Ayres, Sophie Stanton and Dominic Hall.
In 2006, Boulter starred alongside Whishaw in the play Citizenship written by Mark Ravenhill. He appeared on a 2006 episode of the television se